In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("intro_to_pytorch.ipynb")

# **Домашнее задание 4.** Нейронные сети

В этом задании нам предстоит написать небольшую нейронную сеть и обучить ее на датасете [Fashion-MNIST](https://www.kaggle.com/datasets/zalando-research/fashionmnist).

<p style="color: red;">
  Данное задание можно выполнять как в среде с GPU, так и без графического ускорителя — обучение модели не займет много времени.<br><br>
  Перед тем как отправить блокнот с кодом на проверку, установите <code>DEVICE = "cpu"</code> и <code>MAX_EPOCHS = 1</code>.
</p>

In [ ]:
# !pip install kaggle
# !uv pip install kaggle

In [ ]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

from sklearn.metrics import classification_report

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split

from IPython.display import clear_output

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# DEVICE = "cpu"  # раскомментируйте строку перед отправкой блокнота

DEVICE

Давайте скачаем данные с Kaggle и посмотрим на них повнимательнее. Датасет включает обучающую и тестовую подвыборки, каждая из которых представлена отдельным CSV файлом.

Вот краткое описание данных:
- каждая строка представляет отдельное изображение в развернутом виде;
- первый столбец содержит целевую переменную;
- остальные столбцы содержат яркость соответствующих пикселей от 0 до 255 (всего 28 × 28 = 784);

В датасете 10 классов:

0.  T-shirt/top;  
1.  Trouser;  
2.  Pullover;  
3.  Dress;  
4.  Coat;  
5.  Sandal;  
6.  Shirt;  
7.  Sneaker;  
8.  Bag;  
9.  Ankle boot.

In [ ]:
if not os.path.exists("data"):
    !kaggle datasets download -p data/ --unzip zalando-research/fashionmnist

!ls data/

In [ ]:
train_data = pd.read_csv(os.path.join("data", "fashion-mnist_train.csv"))
print(f"{train_data.shape = }")
train_data.head()

In [ ]:
test_data = pd.read_csv(os.path.join("data", "fashion-mnist_test.csv"))
print(f"{test_data.shape = }")
test_data.head()

Отделим признаки от целевых переменных

In [ ]:
X_train, y_train = train_data.drop(columns="label").values, train_data["label"].values
X_test, y_test = test_data.drop(columns="label").values, test_data["label"].values

Отрисуем одно изображение

In [ ]:
print(y_train[2077])

plt.imshow(X_train[2077].reshape(28, 28), cmap="gray")
plt.show()

---

### **Задание 1.** Создание `Dataset` и `DataLoader`

_Points:_ 5

In [ ]:
# TODO: преобразуйте данные в `torch.Tensor`. Приведите признаки к типу данных `torch.float32`
# и отмасштабируйте их, разделив на максимальное значение
...

In [ ]:
# TODO: завершите класс `FashionMNISTDataset`. Полезные ссылки:
# https://docs.pytorch.org/docs/stable/data.html#torch.utils.data.Dataset
class FashionMNISTDataset(Dataset):
    def __init__(self, X: torch.Tensor, y: torch.Tensor) -> None:
        """Инициализирует новый датасет.

        Args:
            X (torch.Tensor): признаки, тензор размера (n, d), где
                n - количество объектов,
                d - количество признаков.
            y (torch.Tensor): целевые переменные, тензор размера (n,).
        """
        super().__init__()

        # TODO: сохраните данные внутри датасета
        ...

    def __getitem__(self, index: int) -> tuple[torch.Tensor, torch.Tensor]:
        """Возвращает один пример (признаки и целевую переменную) по индексу.

        Args:
            index (int): индекс примера.

        Returns:
            tuple[torch.Tensor, torch.Tensor]: кортеж вида (признаки, целевая переменная).
        """
        # TODO: верните необходимые значения
        ...

    def __len__(self) -> int:
        """Возвращает количество примеров в датасете.

        Returns:
            int: размер датасета.
        """
        # TODO: верните размер датасета
        ...

In [ ]:
# TODO: используйте класс `FashionMNISTDataset` для создания `train_dataset`
train_dataset = ...

# TODO: используйте класс `TensorDataset` для создания `test_dataset`. Полезные ссылки:
# https://docs.pytorch.org/docs/stable/data.html#torch.utils.data.TensorDataset
test_dataset = ...

len(train_dataset), len(test_dataset)

In [ ]:
# TODO: отделите 20% от обучающего датасета для валидации. Полезные ссылки:
# https://docs.pytorch.org/docs/stable/data.html#torch.utils.data.random_split
train_dataset, val_dataset = ...

len(train_dataset), len(val_dataset)

In [ ]:
# Создайте `DataLoader` для каждого из датасетов. Используйте `batch_size=256`
train_dataloader = ...
val_dataloader = ...
test_dataloader = ...

In [ ]:
batch = next(iter(train_dataloader))

X, y_true = batch
X.shape, y_true.shape

In [ ]:
grader.check("Task1")

---

### **Задание 2.** Многослойный перцептрон

Напишите двухслойный перцептрон со следующей структурой:

- линейный слой, создающий 256 скрытых признаков;
- нелинейность, например ReLU;
- линейный слой.

При желании после ReLU можно добавить слой [Dropout](`https://docs.pytorch.org/docs/stable/generated/torch.nn.Dropout.html`). Это один из способов регуляризации в нейронных сетях.

_Points:_ 5

In [ ]:
class MLPClassifier(nn.Module):
    def __init__(self, in_features: int, num_classes: int) -> None:
        super().__init__()

        ...

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        ...

In [ ]:
grader.check("Task2")

---

### **Задание 3.** Цикл обучения

Наконец мы готовы приступить к обучению нашей модели.

_Points:_ 5

In [ ]:
MAX_EPOCHS = 50
# MAX_EPOCHS = 1  # раскомментируйте строку перед отправкой блокнота

In [ ]:
# TODO: создайте модель и перенесите ее на `DEVICE`
model = ...

# TODO: создайте оптимизатор, не забудьте передать в него параметры модели и скорость обучения
optimizer = ...

train_loss_history: list[float] = []
val_accuracy_history: list[float] = []

for epoch in range(MAX_EPOCHS):
    model.train()
    train_loss: list[float] = []

    for X, y_true in tqdm(train_dataloader, desc="Training"):
        # TODO: перенесите `X` и `y_true` на `DEVICE`
        X, y_true = ...

        # TODO: сделайте предсказание
        logits = ...

        # TODO: обнулите градиент с прошлого шага
        ...

        # TODO:  вычислите значение функции потерь. Поскольку мы решаем задачу многоклассовой
        # классификации, будем использовать cross entropy loss. Полезные ссылки:
        # https://docs.pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html
        # https://docs.pytorch.org/docs/stable/generated/torch.nn.functional.cross_entropy.html
        loss = ...

        # TODO: вычислите градиент функции потерь относительно параметров модели
        ...

        # TODO: сделайте один шаг обновления весов модели
        ...

        train_loss.append(loss.item())

    train_loss: float = sum(train_loss) / len(train_loss)
    train_loss_history.append(train_loss)

    model.eval()
    val_accuracy: list[float] = []

    with torch.no_grad():
        for X, y_true in tqdm(val_dataloader, desc="Validation"):
            # TODO: перенесите `X` и `y_true` на `DEVICE`
            X, y_true = ...

            # TODO: сделайте предсказание
            logits = ...

            # TODO: определите класс, к которому модель отнесла каждый объект
            y_pred = ...

            # TODO: вычислите долю правильных ответов
            accuracy = ...

            val_accuracy.append(accuracy)

    val_accuracy: float = sum(val_accuracy) / len(val_accuracy)
    val_accuracy_history.append(val_accuracy.item())

    clear_output(wait=True)
    print(f"[{epoch:02d}/{MAX_EPOCHS}] train-loss={train_loss:.4f} | val-accuracy={val_accuracy:.4f}")

In [ ]:
fig, axs = plt.subplots(1, 2)
fig.set_size_inches(12, 4)

axs[0].plot(train_loss_history)
axs[0].set_xlabel("Epoch")
axs[0].set_ylabel("Train Loss")

axs[1].plot(val_accuracy_history)
axs[1].set_xlabel("Epoch")
axs[1].set_ylabel("Val Accuracy")

plt.show()

---

### **Задание 4.** Инференс модели

_Points:_ 5

In [ ]:
model.eval()

all_y_true: list[torch.Tensor] = []
all_y_pred: list[torch.Tensor] = []

with torch.no_grad():
    for X, y_true in tqdm(test_dataloader, desc="Inference"):
        logits = model(X).to(DEVICE)
        y_pred = logits.argmax(dim=-1)

        all_y_true.append(y_true.cpu())
        all_y_pred.append(y_pred.cpu())

all_y_true = torch.cat(all_y_true)
all_y_pred = torch.cat(all_y_pred)

In [ ]:
print(classification_report(all_y_true.numpy(), all_y_pred.numpy()))

In [ ]:
grader.check("Task4")

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False, run_tests=True)